In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import typhon
import xarray as xr
from netCDF4 import Dataset


plt.style.use(typhon.plots.styles('typhon'))

ncpath = '/home/zmaw/u300509/lkluft/icon-aes/psrad/rrtmg_lw.nc'
rrtm_props = Dataset(ncpath)

T_planck = rrtm_props.variables['TemperaturePlanckValues'][:]
T_plot = T_planck.copy()
T_plot = np.append(T_plot, T_plot[-1] + 1)
band_index = np.arange(1, 18)
integrated_planck = rrtm_props.variables['IntegratedPlanckFunction'][:]
band16 = rrtm_props.variables['IntegratedPlanckFunctionBand16'][:]
#integrated_planck[-1, :] = band16

print(integrated_planck.dtype)

fig, ax = plt.subplots()
sm = ax.pcolormesh(T_plot, band_index, integrated_planck, cmap='temperature')
# sm = ax.contourf(integrated_planck, cmap='temperature')
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('Band')
fig.colorbar(sm, label='Integrated Planck values')

In [ ]:
upper_limits = rrtm_props.variables['BandWavenumberUpperLimit'][:]
lower_limits = rrtm_props.variables['BandWavenumberLowerLimit'][:]
T_planck = rrtm_props.variables['TemperaturePlanckValues'][:]

test = np.zeros((band_index.size - 1, T_planck.size))

N = T_planck.size

for i, (bottom, top) in enumerate(zip(lower_limits, upper_limits)):
    wavenumbers = np.linspace(bottom, top, 100) * 100  # wavenumbers in m^-1
    for j, T in enumerate(T_planck):
        planck = typhon.physics.planck_wavenumber(wavenumbers, T + 1)
        test[i, j] = np.trapz(planck, wavenumbers) / (top - bottom) * 1e-4

fig, ax = plt.subplots()
sm = ax.pcolormesh(T_plot, band_index, test, cmap='temperature')
fig.colorbar(sm, label='Integrated Planck values')
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('Band')

fig, ax = plt.subplots()
rel_diff = (test - integrated_planck) / integrated_planck
sm = ax.pcolormesh(T_plot, band_index, rel_diff, cmap='seismic')
cb = fig.colorbar(sm, label='Rel. Diff. Integrated - PSrad')
typhon.plots.center_colorbar(cb)
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('Band')

In [ ]:
# Read band limits
with Dataset(ncpath, 'r') as rrtm_props:
    upper_limits = rrtm_props.variables['BandWavenumberUpperLimit'][:]
    lower_limits = rrtm_props.variables['BandWavenumberLowerLimit'][:]
    # T_planck = rrtm_props.variables['TemperaturePlanckValues'][:]
    # band16 = rrtm_props.variables['IntegratedPlanckFunctionBand16'][:]
    # integrated_planck = rrtm_props.variables['IntegratedPlanckFunction'][:]

# Create new reference temperatures for Planck integration.
T_planck = np.arange(159, 500)

# Pre-allocate new lookup table.
new_lookup = np.zeros((band_index.size - 1, T_planck.size))

for i, (bottom, top) in enumerate(zip(lower_limits, upper_limits)):
    wavenumbers = np.linspace(bottom, top, 50) * 100  # wavenumbers in m^-1
    for j, T in enumerate(T_planck):
        planck = typhon.physics.planck_wavenumber(wavenumbers, T + 1)
        new_lookup[i, j] = np.trapz(planck, wavenumbers) / (top - bottom) * 1e-4

        
# Open the original lookup table.
ds = xr.open_dataset(ncpath)

# Delete the variables we want to change. This allows to alter their shape.
del ds['TemperaturePlanckValues']
del ds['IntegratedPlanckFunction']
del ds['IntegratedPlanckFunctionBand16']

# Write new lookup table to the dataset (including fitting coordinates).
ds['TemperaturePlanckValues'] = T_planck
ds.set_coords('TemperaturePlanckValues')
ds['IntegratedPlanckFunctionBand16'] = xr.DataArray(
    new_lookup[-1, :],
    dims=('TemperaturePlanckValues',),
)
ds['IntegratedPlanckFunction'] = xr.DataArray(
    new_lookup, dims=('Band', 'TemperaturePlanckValues'),
)

# Delete this variable as it can't be stored with xarray.
if 'AbsorberNames' in ds:
    del ds['AbsorberNames']

# Save the Dataset to a **new** netCDF4 file.
# Do not use the original filename! This results in an empty file.
ds.to_netcdf(ncpath + '_modified')